In [1]:
import tweepy
import pandas as pd
import os
import unidecode
#import datetime
from datetime import timedelta, datetime 
from dateutil.relativedelta import relativedelta
import time
import datetime as dt 
import pymongo
from pymongo import MongoClient
import numpy as np
pd.options.mode.chained_assignment = None 

In [6]:
cleaned = pd.read_parquet('../Data/Full_dataset/data_filtered.gzip')

In [6]:
total = len(cleaned)
perc_sample = 0.01
proportion = round(total*perc_sample)
sampled_df = cleaned.sample(proportion, random_state=1)
sampled_df = sampled_df.reset_index(drop=True)

In [2]:
sel = pd.read_csv('../Data/SEL/SEL_full.txt', delimiter='\t', encoding= 'ISO-8859-1')
sel  = sel.drop_duplicates(subset=['Palabra', 'Categoría'])
sel_list = list(sel['Palabra'])

In [20]:
emotions_dict = {}
for i in sel['Categoría'].unique():
    emotions_list = list(sel[sel['Categoría'] == i].sort_values(by= [' PFA'], ascending = False)[:20]['Palabra'])
    emotions_dict[i] = emotions_list

In [23]:
emotions_words = pd.DataFrame(emotions_dict)
emotions_words.to_csv('../Data/Sidorov/top_emotions_words.csv', index = False)

In [16]:
alegria_list = sel[sel['Categoría'] == 'Alegría'].sort_values(by= [' PFA'], ascending = False)[:20]['Palabra']

In [298]:
cleaned['tweet_split'] = cleaned['text'].str.replace('[^\w\s]','').str.lower()
cleaned['tweet_split'] = cleaned['tweet_split'].str.split()
emociones = list(sel['Categoría'].unique())
for i in emociones:
    cleaned[i] = 0.0

/tmp/ipykernel_5036/1904514863.py:1: FutureWarning: The default value of regex will change from True to False in a future version.
  cleaned['tweet_split'] = cleaned['text'].str.replace('[^\w\s]','').str.lower()


In [318]:
def emotional_values(df):
    text_list = df['tweet_split']
    text_length = df['Number_words']
    emo_values = dict.fromkeys(emociones, 0)
    for i in text_list:
        if i in sel_list:
            for j in sel[sel['Palabra'] == i]['Categoría']:
                emo_values[j] += float(sel[(sel['Palabra'] == i) & (sel['Categoría'] == j)][' PFA'])/text_length
    df[emociones] = [emo_values[i] for i in emociones]
    return(df)

In [319]:
start_time = time.time()
cleaned_emotions = cleaned.apply(emotional_values, axis=1)
print("--- %s seconds ---" % (time.time() - start_time))

--- 746.3502352237701 seconds ---


In [292]:
emociones

['Alegría', 'Enojo', 'Miedo', 'Repulsión', 'Sorpresa', 'Tristeza']

In [332]:
cleaned_emotions['Emotion'] = cleaned_emotions[emociones].idxmax(axis=1)
cleaned_emotions['Emotion_prob'] = cleaned_emotions[emociones].max(axis=1)
cleaned_emotions['Emotion'] = np.where(cleaned_emotions['Emotion_prob'] == 0.0, 'Otra', cleaned_emotions['Emotion'])

In [356]:
cleaned_emotions.to_parquet('../Data/Full_dataset/data_filtered_emotions.gzip',
          compression='gzip')
cleaned_emotions['Emotion'].value_counts()

Otra         167752
Alegría       63234
Tristeza      24350
Enojo         12195
Miedo         10381
Repulsión      6896
Sorpresa       6027
Name: Emotion, dtype: int64

In [364]:
emotions_df = []
for i in emociones:
    df = cleaned_emotions[cleaned_emotions['Emotion'] == i].sort_values(by=['Emotion_prob'], ascending=False)[['text', 'Emotion', 'Emotion_prob']].iloc[0:2000]
    emotions_df.append(df)
full_emotions_df = pd.concat(emotions_df)

In [367]:
full_emotions_df.to_parquet('../Data/Emotional_tweets/raw_emotional_tweets.gzip',
          compression='gzip')